In [6]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [7]:
response = requests.get(
    "https://sea.wildstats.gg/en/profile/tlBhK5Y71Xm1qjXblmwQ/battle/898783879589689163")

In [8]:
soup = BeautifulSoup(response.text, "html.parser")

row_elements = soup.findAll('td', attrs = {'class' : 'text-grey'})

rows = [row_element.text.strip() for row_element in row_elements]

column_elements = soup.findAll(attrs={"class":"m-auto"})

columns = [column_element.get("title").split()[2] for column_element in column_elements]

df = pd.DataFrame(columns=columns, index=rows)

texts = soup.findAll(attrs={"class":"text-center", "data-toggle":"tooltip"})

all_texts = [text.text.strip() for text in texts]
all_texts = np.array(all_texts[10:]).reshape((-1, 10))

for i, texts in enumerate(all_texts):
  for j, text in enumerate(texts):
    if text == "-":
      all_texts[i, j] = '0'
    elif text == "✔":
      all_texts[i, j] = '1'
    elif "k" in text:
      pass
    elif "%" in text:
      all_texts[i, j] = text.replace("%", "")

all_texts = np.char.replace(all_texts, 'k', '000')
all_texts = all_texts.astype(np.int32)
all_texts = np.insert(all_texts, 0, [1, 2, 3, 4, 5]*2, 0)
df = pd.DataFrame(all_texts, columns = columns, index = rows)

In [9]:
print(df.corrwith(df.loc["Game Points"], axis=1).sort_values())

Deaths                                     -0.901543
Minions Killed                             -0.533368
Health Recovered from honeyfruits          -0.514257
Joined Baron Kills                         -0.463525
Krug Kills                                 -0.372008
Active Skill Equipped                      -0.312625
Total Honeyfruit Used                      -0.292084
Low Power Kill Number                      -0.274786
Emoji                                      -0.258534
Hang Time                                  -0.255503
Honeyfruits Picked                         -0.242541
Herald Tower Kills                         -0.216936
Max Trail Kill Number                      -0.180583
Quick Chats Sent                           -0.166318
Total Healing Received                     -0.123225
Hits using Skill 2                         -0.098531
Seconds to Call Rift Herald                -0.062670
Damage to Champions Dealt with Abilities   -0.040427
Scryer Blooms Used                         -0.

In [10]:
print(df.corrwith(df.loc["Game Points"], axis=1).sort_values(key = lambda x: abs(x)))

Damage Taken by Champions                  -0.002476
Hits using Skill 1                         -0.003567
Enemies Controlled                          0.005046
Total Damage Dealt by Abilities             0.013486
Scryer Blooms Used                         -0.019666
Rejected Surrender                          0.021694
Total Damage dealt by Attacks               0.034735
Total Wards                                 0.034890
Damage to Champions Dealt with Abilities   -0.040427
Seconds to Call Rift Herald                -0.062670
Skill 4 Count                               0.068488
Total Damage Tanked                         0.096188
Hits using Skill 2                         -0.098531
Total Damage Dealt                          0.104798
Signals                                     0.110349
Red Buff Kills                              0.111881
Total Healing Received                     -0.123225
Hits using Skill 4                          0.125736
Seconds to First Blood                      0.

In [11]:
x = df.drop(['Seconds in Game', 'Game Points', 'Team MVP']).to_numpy()
y = df.loc["Game Points"].to_numpy()

reg = LinearRegression().fit(np.rot90(x), y)

print(x.shape, y.shape)
reg.score(np.rot90(x), y)

(89, 10) (10,)


1.0

In [12]:
reg.coef_

array([-1.16172904e-06, -5.77122457e-08,  3.36336432e-04,  4.37875221e-07,
        1.94147561e-07, -2.41048701e-06,  7.85995160e-08, -1.63949538e-07,
       -3.53169965e-07,  2.63831837e-07,  1.65061417e-07, -3.20540514e-07,
        2.10359492e-07, -7.87095613e-04, -7.71076302e-04, -5.76072427e-04,
       -4.67480329e-04,  1.70818014e-05, -2.44162039e-04, -1.37097586e-03,
        7.16387403e-04, -9.20252856e-04, -5.40461604e-04, -4.86896907e-08,
       -2.44809832e-07,  1.49670955e-07, -7.36797382e-06,  2.79219434e-06,
       -1.47975892e-05,  6.99322286e-05, -3.53169965e-07, -7.93238863e-04,
       -2.03675108e-04, -1.37640452e-03,  8.00361100e-07,  1.38406298e-06,
       -3.03890928e-05,  3.48256800e-07, -9.32696529e-05, -2.24205896e-07,
        1.12916111e-04,  1.22073727e-06,  1.86495706e-07,  1.94357655e-07,
       -4.27148475e-07,  1.37884531e-06, -3.07316186e-07,  2.29695313e-06,
       -2.59439076e-08, -9.27730714e-07, -1.47290692e-06,  2.22962025e-06,
        2.74697693e-07, -